To complete the assignment this week you do need to have a good grasp of the python mocking framework provided by the unittest.mock module.  As I said in the lecture, each language has it's own frameworks and their own way of doing things, and I didn't necessarily want to focus too much attention on teaching a specific technology, as that will change.  It is important to understand the basic concepts of mocking, which should be consistent across implementations.  So I hope in this note to describe the concepts, using the python mocking framework to help solidify the ideas.

You need to understand a few things.  #1 - what is special about a mock object.  #2 - how python allow you to change the way a program works by swapping substituting a mock in place of a function or an object, and #3 - how #1 and #2 together allows you to construct a unittest that gives you full control over the test and totally isolates your test from any external dependencies.


# 1 - why are mocks special?
Before we look at the mock object, let's recall something important about python objects in general.  Remember that you can create a python object from a class.  Let's create the simplest class called "Foo" and then create an instance of the class and we will call it f:

In [2]:
class Foo:
      pass

f = Foo()

Now what can we do with f?   Well actually, not much.  If we try to access an attribute on f we will get an error.  That is because we've defined no attributes on f:

In [3]:
f.someAttribute

AttributeError: 'Foo' object has no attribute 'someAttribute'

And what if we try to invoke a method on f?  Well we will also get an error, this is because we have not defined any methods on f:

In [4]:
f.someMethod()

AttributeError: 'Foo' object has no attribute 'someMethod'

It also returns an error.
The class Foo and its associated instance f doesn't have much magic.  It can't do much of anything.

So let's compare this to the Mock class and instances of Mock.

A mock object is created by calling the Mock() constructor which is imported from the unittest.mock package.  You can use either Mock or MagicMock as your mock class.  For our example it doesn't matter, but MagicMock has more "magic" so I tend to use that.  SO you can import using either (a) or (b), and as I said, I prefer (b) but it wouldn't matter for this example.

# (a)
from unittest.mock import Mock

# (b)
from unittest.mock import MagicMock as Mock

Now you can create a instance of the mock class by doing this:


In [5]:
from unittest.mock import MagicMock as Mock

In [6]:
m = Mock()


So what is so special about this instance m?  For one, we can try to access any attribute on this object and it will not return an error:

In [7]:
m.someAttribute

<MagicMock name='mock.someAttribute' id='1712583021456'>

Likewise, we can call any method on m and it will not return an error:

In [8]:
m.someMethod()

<MagicMock name='mock.someMethod()' id='1712583117288'>

we can even pass arguments to the method:

In [9]:
m.someMethod(f)

<MagicMock name='mock.someMethod()' id='1712583117288'>

But so what!  What else can it do?   Well for one, it can allow you to tell the mock object what to return when you try to access an attribute or when you try to call an object.  See here how I can set a value on the attribute of the mock, and then when I try to access teh attribute then the value is returned:

In [10]:
m.someAttribute = 420

In [11]:
m.someAttribute

420

What about method calls on the mock?  Can I control what the method will return when I call it?  Well sure, take a look here. We can do that by setting the "return_value" attribute of the function:

In [12]:
m.someFunction.return_value = "hello"

Now when I call the function on the mock instance, see what is returned:

In [13]:
m.someFunction()

'hello'

The way all this magic works under the covers isn't important right now (but I encourage you to take a deeper dive to look into that), but you should understand that when you access the attributes of the mock or call functions on the mock that it will by default return an instance of a mock.

For example


In [14]:
m.anotherAttribute

<MagicMock name='mock.anotherAttribute' id='1712583142760'>

likewise:

In [15]:
m.anotherMethod()

<MagicMock name='mock.anotherMethod()' id='1712583201456'>

So you can in fact "chain" operations on the mock object.

In [16]:
m.anotherAttribute.someOtherAttribute.anotherMethod().someAttribute = 999

In [17]:
m.anotherAttribute.someOtherAttribute.anotherMethod().someAttribute

999

In [18]:
m.anotherAttribute.someOtherAttribute.anotherMethod().aFunction.return_value = "goodbye"

In [19]:
m.anotherAttribute.someOtherAttribute.anotherMethod().aFunction()

'goodbye'

There are in fact many more other things that we can do with mock objects which I won't cover here, but one other one that is important for this assignment is the side_effect attribute.   Remember before that we could set the return value of a method on a mock:

In [20]:
m.someMethod.return_value = "goodbye"

So that whenever we called m.someMethod() it would return "goodbye".  It would do this everytime we called it:


In [21]:
print(m.someMethod())
print(m.someMethod())
print(m.someMethod())

goodbye
goodbye
goodbye


but what about if we wanted that method to return a different result each time we called it?   Say we wanted it to return "good morning" the first time, then return "good afternoon" the second time, and then finally return "good night" the third time?   Well we can achieve that using the "side_effect" attribute:

In [22]:
results = [Mock(), Mock(), Mock()]
results[0] = "good morning"
results[1] = "good afternoon"
results[2] = "good night"

m.greeting.side_effect = results

Now when we call the greeting() method we get a different result each time

In [23]:
print(m.greeting())
print(m.greeting())
print(m.greeting())

good morning
good afternoon
good night


If we call it too many times, however, we get an error:

In [24]:
print(m.greeting())

StopIteration: 

If you understand all this, then you're in a good position to understand the next part, namely how the Mocking framework in Python allows you to inject mock objects into a program, wherever you want.

# 2 - how python allows you to change the way a program works by substituting a mock in place of a function or an object

The unittest.mock module provides a handy method (a decorator as well) that can be used to modify a program, or, in other words, to "patch" the program.   So say, for example, that we have a function "calc"() that does calculations, but that the function uses a cloud service to do the computation.  The Cloud class implements this very expensive function called "expensive_operation".   Whenever you run this it takes 5 seconds to do the calculation.

In [25]:
from time import sleep

class Cloud:
    def expensive_operation(self, a,b):
        print("starting expensive operation")
        t = 3
        sleep(t)
        print(F"finished expensive operation {t} seconds later")
        return a + b
    
def calc(x,y):
    cld = Cloud()
    z = cld.expensive_operation(x,y)
    return z

With this defintion, whenever we call "calc()" the method will invoke the Cloud method called "expensive_operation" (say it calls into some service in the cloud which uses a lot of memory & compute and which costs many dollars) and then returns the result.  For example:


In [26]:
calc(12,2)

starting expensive operation
finished expensive operation 3 seconds later


14

Let's say that the dev team is writing some code to call the calc() method.  

In [27]:
def dev_teams_playing_around():
    x = 5
    y = 2
    z = calc(x,y)
    return z

when the dev teams are playing around, the calculations take a long time and are expensive.  What we want to do is change the program so that we don't call the expensive_operation, but rather we'd want to substitute it with a less expensive operation.

The problem is that we're not allow to change the Cloud class or the calc() method.  We need to somehow change how this program behaves, without changing the program.   How can we do that?

What we can do is "patch" the program to achieve this. By using "patch," we can change the methods that a program calls, or we can change the objects that are referenced by the program, or we can change the classes referenced by the program, without changing one line in the program.

Let's look at some examples of how patch can be used to achieve this.

First let's define the cheaper operation.  THis is what we want the developers to use when they're playing around vs. the more expensive operation:


In [28]:
def cheap_operation(_, a,b):
    print("cheap operation")
    return a + b

Next we need to import patch from the mock module

In [29]:
from unittest.mock import patch

Then we can use "patch" to change the operation from an expensive operation to a cheaper one.
First the developers run the program without patching:


In [30]:
dev_teams_playing_around()

starting expensive operation
finished expensive operation 3 seconds later


7

Then the developers patch the program and then run it again:

In [31]:
with patch.object(Cloud, 'expensive_operation', new=cheap_operation):
    x = dev_teams_playing_around()
    print(x)
    


cheap operation
7



What this did was patch the method Cloud.expensive_operation with a new method called "cheap_operation", so that when the developers call dev_teams_playing_around(), that deep down the calc() method ultimately called the cheap_operation instead of the expensive_operation.

Patch also allows one to use a python decorations to patch the program.   For example, let's define another method for the developers playing around, but this one will be patched


In [32]:
@patch('__main__.Cloud.expensive_operation', cheap_operation)
def dev_team_playing_patched():
    return calc(1,2)

now if we call "dev_team_playing_patched()" you'll see that it will call the cheap operation.

In [33]:
dev_team_playing_patched()

cheap operation


3

So you might be asking, what does this have to do with mocks?  
The answer is that patch also has some magic in it.  

When you use @patch and you indicate that you want to patch a method some object, then @patch will do two things:
1. @patch will create an instance of a Mock and use the mock instead of the method or the object that you're patching.   In other words, if we say that we want to patch "expensive_operation" but we don't tell patch what to replace it with (remember before, we told patch to replace expensive_operation with cheap_operation), then patch will replace expensive_operation with a mock.  It means that when the calc() method calls "expensive_operation" that it will be calling a mock, which as we know will just return another mock.

We can see that here:

In [34]:
@patch('__main__.Cloud.expensive_operation')
def dev_team_playing_patched_with_mock(_):
    return calc(1,2)

dev_team_playing_patched_with_mock()


<MagicMock name='expensive_operation()' id='1712583633160'>

The second thing that @patch does is:

2. After @patch creates the mock, it will inject it into the method that @patch is decorating.  In our example here, @patch will add an argument to the end of the method "dev_team_playing_patched_with_mock" so that that method now has access to the mock object.  Now you can do things to the mock to change how it will behave whe it is called from within "calc()".

In the example here, the mock is injected into the method and I named the mock "injectedMock".  Now that we have that mock we can set its return value, so that when it is called it will return whatever we want, instead of just returning itself as a mock.  See here:


In [35]:
@patch('__main__.Cloud.expensive_operation')
def dev_team_playing_patched_with_mock(injectedMock):
    injectedMock.return_value = 3
    return calc(1,2)

Now when we call this method: dev_team_playing_patched_with_mock() we will see that it returns 3

In [36]:
dev_team_playing_patched_with_mock()

3

At this point you should hopefully understand what mock objects are, and why they are special, as well as how we can use patch to change a program, without having to change the code directly.   With those two pieces we can now take a look to see how we can use this to mock unit tests.

# 3 - how 1 and 2 together allows you to construct a unittest that gives you full control over the test and totally isolates your test from any external dependencies


Say that you have a program that does something and you want to write a unit test for that program.  Also say that the progran accesses some external services which are unreliable (think GitHub APIs that have a limit).   What you can do is write a unit test so that it will patch the program that you're trying to test and substitute a mock for all those methods that access external services.
Then in your unit test you will get a handle on that mock object (it will be passed in as an argument), and there you will be able to configure what that object returns, so that instead of your program calling a GitHub API and returning a result, the program will call your mock object and it will return whatever you have set it up to return.

For example, say the program does the following:

    def get_commits_count(user):
        ...
        # setup the GITHUB URL with the user
        ...
        # use requests to call github
        repos = requests.get(GITHUB_REPOS_URL_WITH_USER)
        repos_json = repos.json()
        result = []
        for repo in repos_json:
            repo_name = repo['name']
            # setup the GITHUB URL with the user & repos
            commits = requests.get(GITHUB_COMMITS_URL_WITH_USER_AND_REPO)
            commits_json = commits.json()
            # get the number of commits from the result
            num_commits = len(commits_json)
            ...
            result.append((repo, num_commits))
        ...
        return result

and our unit test looks like this:

    def test_get_commits_count():
        commit_count_list = get_commits_count(USER)
        asset_equal(commit_count_list, [('repo1', 10), ('repo2', 45)])

The problem is that the GitHub API is unreliable, so we want to mock out the call to requests.get

We can start this as follows:

    @patch('requests.get')
    def test_get_commits_count(injectedMock):
        commit_count_list = get_commits_count(USER)
        asset_equal(commit_count_list, [('repo1', 10), ('repo2', 45)])

remember that what this will do is create the Mock() object and inject it, where in this case the mock object is named 'injectedMock', so that the test method will have access to an instance of Mock() just as if it was created as so:

In [63]:
injectedMock = Mock()

we did before:  we used @patch to patch the requests.get method with a mock, and that mock will be injected into the unit test method named "injectedMock"

What we need to do is set up the injecteMock mock object so that it returns whatever results are expected, when it is called. Note that it will be called multiple times, so we will need to use the "side_effect" attribute, as we discussed above.

So think about it... the first time requests.get() is called, it will return an object, and then immediately after that the json() method will be called on that object, and that we expect the result of calling json() to be a JSON object.  That JSON object will be a list of JSON objects, and each of those objects will have an element called "name" that will be the name of a repo.

That would be something that looks like this:

In [64]:
json_str = '[ { "name" : "repo1" }, { "name" : "repo2" } ]'

but this is a string, not a JSON object.  We need to turn this string into a JSON object.

So how do we turn a string that looks like JSON into a JSON object?   Use the json.loads() method!


In [65]:
import json

json_obj = json.loads(json_str)


Now that we have a json object, we want to set up the mock so that when when someone calls the json() method on the mock, that this json object is returned.

If our mock object was just returning one value, we could do this:



In [66]:
    m = Mock()
    m.json.return_value = json_obj

then when we called m.json() it would return json_obj


In [86]:
m.json()

[{'name': 'repo1'}, {'name': 'repo2'}]

But we know that our mock will be called multiple times, because we know that requests.get will be called multiple times.
So we have to set up side_effect instead, as follows:


In [89]:
# assume that here we are in the body of the test method, and injectedMock as injected as a parameter to the method:
    
results = [Mock(), Mock(), Mock()]
results[0].json.return_value = json.loads('[ { "name" : "repo1" }, { "name" : "repo2" } ]')
results[1].json.return_value = json.loads('[ { "commit" : "abc" }, { "commit" : "blah" }, \
                                          { "commit" : "baz" }, { "commit" : "foo" } ]')
results[2].json.return_value = json.loads('[ { "commit" : "abc" }, { "commit" : "blah" } ]')
injectedMock.side_effect = results

assume that we are patching the method called "req_get", then what patch will do is set req_get to be equal to the injectedMock. 



In [90]:
req_get = injectedMock

Then when the method we mocking, req_get, is called, multiple times, and then for each when we call the json() methon on the result, then it will return multiple JSON objects, in turn, just as you have them set up.

In [91]:
j = req_get()
j.json()

[{'name': 'repo1'}, {'name': 'repo2'}]

In [92]:
j = req_get()
j.json()

[{'commit': 'abc'}, {'commit': 'blah'}, {'commit': 'baz'}, {'commit': 'foo'}]

In [93]:
j = req_get()
j.json()

[{'commit': 'abc'}, {'commit': 'blah'}]

Please have a good look at this sheet and let me know if you any questions, or if you detect any errors.  You know how to reach me.   Good luck.
